In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import warnings
tf.set_random_seed(1)
np.random.seed(1)
warnings.filterwarnings("ignore")# 关掉警告，真是太棒了 
# 关掉警告第二次有效，但是这个程序只能不断重新开始kernel，否则会因为rnn的命名产生错误
# 两个出现矛盾了
# 代码是按照python深度学习基于tensorflow的第十五章LSTM写的

In [ ]:
# 载入数据集
mnist = input_data.read_data_sets("E:/tensorflow/MNIST_data",one_hot=True)
# 查看数据集大小
print(mnist.train.images.shape)
print(mnist.train.labels.shape)
print(mnist.test.images.shape)
print(mnist.test.labels.shape)

In [ ]:
# 可视化其中一张图片
plt.imshow(mnist.train.images[0].reshape((28,28)),cmap='gray')
plt.title('%i'% np.argmax(mnist.train.labels[0]))
plt.show()

In [ ]:
# 定义超参数
BATCH_SIZE = 64
TIME_STEP = 28 # RNN的时间步，一张图片需要读取28行 高度
INPUT_SIZE = 28 # RNN的输步长，每次读取的向量长度，即每行由28个像素点构成 宽度
LR = 0.01 # 定义学习率Learning Rate

# 读入数据
test_x = mnist.test.images[:2000]
test_y = mnist.test.labels[:2000]

#定义表示x向量的tensorflow placehoders
tf_x = tf.placeholder(tf.float32,[None,TIME_STEP*INPUT_SIZE])# 形状为(batch,784)
image = tf.reshape(tf_x,[-1,TIME_STEP,INPUT_SIZE])
# 定义表示y向量的tensorflow placehoders
tf_y = tf.placeholder(tf.int32,[None,10])

# RNN的循环结构体，使用LSTM结构
rnn_cell = tf.contrib.rnn.BasicLSTMCell(num_units=64)
# 构建网络
outputs,(h_c,h_n) = tf.nn.dynamic_rnn(
    rnn_cell,              # LSTM cell
    image,                 # 输入图片
    initial_state = None, # 隐藏层的初始化状态
    dtype = tf.float32,    # 如果initial_state = None,dtype必须给定
    time_major = False    # TIME_STEP是否在输入数据的第一个维度  
    # Fasle:(batch,time_step,input)  True:(time_step,batch,input)
)
# 取最后一个时刻的输出作为读完每张图片后的最后输出
output = tf.layers.dense(outputs[:,-1,:],10)
# 计算损失函数
loss = tf.losses.softmax_cross_entropy(onehot_labels=tf_y,logits=output)# 使用交叉熵损失
# 定义训练过程
train_op = tf.train.AdamOptimizer(LR).minimize(loss)
# 计算预测精度
accuracy = tf.metrics.accuracy(labels=tf.argmax(tf_y,axis=1),predictions=tf.argmax(output,axis=1))[1]
sess = tf.Session()
# 初始化精度计算中所需要的变量
init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
# 初始化计算图中的变量
sess.run(init_op)

# 开始训练
for step in range(1200):
    b_x,b_y = mnist.train.next_batch(BATCH_SIZE)
    _,loss_ = sess.run([train_op,loss],{tf_x:b_x,tf_y:b_y})
    if step % 50 == 0: # 计算精度
        accuracy_ = sess.run(accuracy,{tf_x:test_x,tf_y:test_y})
        print('train loss:%.4f' % loss_,'| test accuracy: %.2f' % accuracy_)
# 输出测试集中的十个预测结果
test_output = sess.run(output,{tf_x:test_x[:10]})
pred_y = np.argmax(test_output,1)
print(pred_y,'prediction number')
print(np.argmax(test_y[:10],1),'real number')